<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:right; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

</center>

# [Certificat Science des Données](https://github.com/Certificat-sciences-des-donnees-bigdata) [Module de Sensibilisation](https://github.com/Certificat-sciences-des-donnees-bigdata/Module-sensibilisation)

# [Introduction à l'Apprentissage Statistique]()

# Adaptation Statistique d'un Modèle de Prévision du Pic d'Ozone

**Résumé**: Exploration puis modélisation de données climatiques en utilisant Python et la librairie [Scikit-learn](http://scikit-learn.org/stable/#). L'objectif est de prévoir pour le lendemain un possible dépassement d'un seuil de concentration en ozone à partir d'une prévision déterministe sur un maillage grossier et de variables climatiques locales. Estimation par différentes méthodes: régression [logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf), [k plus proches voisins](http://wikistat.fr/pdf/st-m-app-add.pdf), [arbre de décision](http://wikistat.fr/pdf/st-m-app-cart.pdf), [agrégation de modèle](http://wikistat.fr/pdf/st-m-app-agreg.pdf), [SVM](http://wikistat.fr/pdf/st-m-app-svm.pdf). Comparaison des [erreurs de prévision](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf) sur un échantillon test puis des courbes ROC. Itération sur plusieurs échantillons tests pour analyser la distribution de l'erreur de prévision. 

 <FONT COLOR="Green">
**Ce tutoriel est divisé en trois parties: les deux du cours  du module Sensibilisation plus des compléments. **
1. Exploration
2. Apprentissage et prévision
3. Compléments correspondant au module Immersion
</font>

## 1. Introduction

L'objectif, sur ces données, est d'améliorer la prévision déterministe (MOCAGE), calculée par les services de MétéoFrance,  de la concentration d'ozone dans certaines stations de prélèvement.  Il s'agit d'un problème dit d'*adaptation statistique* ou post-traitement d'une prévision locale de modèles à trop grande échelle en s'aidant d'autre variables également gérées par MétéoFrance, mais à plus petite échelle (température, force du vent...). 

La question posée reste: quelle est la meilleure stratégie pour prévoir l'occurrence d'un pic de pollution. 

Comme avec R différentes méthodes sont testées : régression logistique, k plus proches voisins, arbre de décision, random forest, SVM. Les réseaux de neurones nécessitent la version 0.18 de Scikit-learn. De façon générale on suppose que l'utilisatuer dispose d'une installation python à jour. Le calepin a été testé avec la version 3 de python.

# <FONT COLOR="Green">Première partie: Exploration</font>

## 2. Prise en compte des données

Les données ont été extraites et mises en forme par le service concerné de Météo France. Elles sont décrites par les variables suivantes:


* **O3obs** La concentration d'ozone effectivement observée le lendemain à 17h locales correspondant souvent au maximum de pollution observée ; (CIBLE)
* **jour** Le type de jour ; férié (1) ou pas (0) ;
* **MOCAGE** Prévision de cette pollution obtenue par un modèle déterministe de mécanique des fluides (équation de Navier et Stockes);
* **TEMPE** Température prévue par MétéoFrance pour le lendemain 17h ;
* **RMH2O** Rapport d'humidité ;
* **NO2** Concentration en dioxyde d'azote ;
* **NO** Concentration en monoxyde d'azote ;
* **station** Lieu de l'observation : Aix-en-Provence, Rambouillet, Munchhausen, Cadarache et Plan de Cuques ;
* **VentMOD** Force du vent ;
* **VentANG** Orientation du vent. 

Ce sont des données "propres", sans trous, bien codées et de petites tailles. Elles présentent avant tout un caractère pédagogique.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Lecture des données présentes dans le dépôt: "./Data/"
df = pd.read_csv(
    "https://www.math.univ-toulouse.fr/~besse/Wikistat/Data/depSeuil.dat", 
    sep = ",",
    header = 0,
    dtype = {"JOUR": "category", "STATION": "category"}
)

df.head()

In [ ]:
df.describe()

## 3. Exploration

Même si les données ne présentent pas de défauts particuliers, une étude exploratoire préliminaire est indispensable afin de s'assurer le leur bonne cohérence, proposer d'éventuelles transformations et analyser les structures de corrélations ou plus généralement de liaisons entre les variables, de groupes des individus ou observations. Il est aussi intéressant d'identifier les observations anormales (outliers).

### 3.1 Unidimensionnelle

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df["O3obs"].hist()

In [ ]:
df["MOCAGE"].hist()

In [ ]:
import math

In [ ]:
df["SRMH2O"] = np.sqrt(df["RMH2O"])
df["LNO2"]   = np.log(df["NO2"])
df["LNO"]    = np.log(df["NO"])

df = df.drop(["RMH2O", "NO2", "NO"], axis = "columns")

**Exercice:** Visualisez les variables `[SRMH2O, LNO2, LNO]` avec un histogramme et commentez la structure de ces variables que nous avons créées. Quelle est l'impact de cette transformation pour un modèle linéaire?

Supprimer les variables explicatives initiales `[SRMH2O, LNO2, LNO]` et construire ci-dessous la variable `dépassement de seuil` pour obtenir le fichier qui sera effectivement utilisé.

In [ ]:
df["dep_seuil"] = df["O3obs"] > 150
df.head()

### 3.2 Exploration multidimensionnelle

In [ ]:
columns = ["O3obs", "MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]

pd.plotting.scatter_matrix(
    df[columns], 
    alpha   = 0.2, 
    figsize = (15, 15), 
    diagonal = 'kde'
)

plt.show()

**Question:** Commenter les relations entre les variables prises 2 à 2.

### 3.3 [Analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
# Réduction des variables
columns = ["O3obs", "MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]

X = scale(df[columns])

Les commandes suivantes permettent de réaliser une analyse en composantes principales sur les seules variables quantitatives. Par ailleurs la variable à modéliser (O3obs, concentration observée) n'est pas utilisée.

In [ ]:
# Calcul des composantes principales:
pca = PCA()
C = pca.fit(X).transform(X)

In [ ]:
# Variance expliquée / composantes principales:
plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
# Distribution des composantes principales:
plt.boxplot(C[:,0:20])
plt.show()

**Q** Commenter ces résultats: quel choix de la dimension? 

**Q** Présence de valeurs atypiques.

In [ ]:
# Repésentation des individus:

plt.figure(figsize=(5,5))

for i, j, nom in zip(C[:,0], C[:,1], df["dep_seuil"]):
    color = "red" if nom  else "blue"
    plt.plot(i, j, "o", color = color)
    
plt.axis((-4,6,-4,6))  
plt.show()

In [ ]:
## coordonnées et représentation des variables
coord1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(1, 1, 1)

for i, j, nom in zip(coord1, coord2, columns):
    
    plt.text(i, j, nom)
    plt.arrow(0, 0, i, j, color = 'black')
    
plt.axis((-1.2, 1.2, -1.2, 1.2))

# cercle
c = plt.Circle((0,0), radius = 1, color = 'gray', fill = False)
ax.add_patch(c)
plt.show()

**Question:** Quel est la structure de corrélation des variables?

**Question:** Une séparatrice linéaire (hyperplan) semble-t-elle adaptée à l'identification des classes?    

Ce n'est pas utile ici mais une classification non supervisée est facile à obtenir à titre illustratif, par exemple en 3 classes, par l'algorithme k-means:

In [ ]:
from sklearn.cluster  import  KMeans
from  sklearn.metrics import confusion_matrix

In [ ]:
clustering = KMeans(n_clusters=3)
clustering.fit(X)

Les classes attribuées aux différentes observations:

In [ ]:
classes = clustering.labels_
classes

In [ ]:
# Repésentation des individus dans les coordonnées de l'ACP.
plt.figure(figsize=(10,8))
plt.scatter(C[:,0], C[:,1], c = classes) 
plt.show()

# <FONT COLOR="Green">Deuxième partie: Apprentissage et Prévision</font>

## 4. Modélisations

La recherche  d'une meilleure méthode de prévision suit généralement le protocole suivant dont la première étape est déja réalisée.

1. Etape descriptive préliminaire uni et multidimensionnelle visant à repérer les incohérences, les variables non significatives ou de distribution exotique, les individus non concernés ou atypiques... et à étudier les structures des données. Ce peut être aussi la longue étape de construction de variables, attributs ou *features* spécifiques des données. 

2. Procéder à un tirage aléatoire d'un échantillon *test* qui ne sera utilisé que lors de la *dernière étape* de comparaison des méthodes.

3. La partie restante est l'échantillon d'*apprentissage* pour l'estimation des paramètres des modèles.

4. Pour chacune des méthodes, optimiser la complexité des modèles en minimisant une estimation "sans biais" de l'erreur de prévision, par exemple par [*validation croisée*](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf).
    - Variables et interactions à prendre en compte dans la régression linéaire ou logistique;
    - variables et méthode pour l'analyse discriminante;
    - nombre de feuilles dans l'arbre de régression ou de classification;
    - architecture (nombre de neurones, pénalisation) du perceptron;
    - algorithme d'agrégation, 
    - noyau et pénalisation des SVMs.
    
5.  Comparaison des qualités de prévision sur la base du taux de mal classés pour le seul échantillon test qui est resté à l'écart de tout effort ou "acharnement" pour l'optimisation des modèles.

**Remarques**

* Lorsque vous disposez d'un petit volume de données, il est recommandé d'utiliser des procédures de validation adaptées afin de réduire la variance (moyenne) des estimations des erreurs de prévision. Nous utiliserons la [validation croisée](https://fr.wikipedia.org/wiki/Validation_croisée) dans le cadre de cette étude. Vous trouverez les détails de cette procédure à la fin de ce cahier.

* Le critère utilisé lors de la validation dépend du problème: erreur quadratique, taux de mauvais classement, AUC (aire sous la courbe ROC), indice de Pierce, *log loss function*...

### 4.1 Extraction des échantillons apprentissage et test

In [ ]:
df.head()

Transformation des données pour l'apprentissage. 

**Q** Pourquoi les variables qualitatives sont-elles transformées en paquets d'indicatrices ou *dummy variables*?

**Q** Pourquoi le type data frame est transformé en une matrice. 

In [ ]:
# Variables explicatives
df = pd.get_dummies(df, columns = ["JOUR", "STATION"])
df.head()

In [ ]:
# Variable cible au format booléen:
y_binary = df["dep_seuil"].copy()

# Variable cible au format continu:
y = df["O3obs"].copy()

In [ ]:
# On retire les variables i.e O3obs et dep_seuil du dataframe car nous ne devons pas utiliser les variables
# cibles pour entrainer les algorithmes. Nous stockerons la variable O3obs pour l'étudier ensuite.
df = df.drop(["JOUR_0", "dep_seuil", "O3obs"], axis = 'columns')

In [ ]:
y.hist()

Extractions des échantillons d'apprentissage  et test pour les deux types de modèles. Comme le générateur est initalisé de façon identique, ce sont les mêmes échantillons dans les deux cas.

In [ ]:
from sklearn.model_selection import train_test_split  

In [ ]:
X_train, X_test, y_binary_train, y_binary_test = train_test_split(
    df, 
    y_binary, 
    test_size = 200, 
    random_state = 42
)

y_train = y.loc[y_binary_train.index]
y_test  = y.loc[y_binary_test.index]

L'étape suivante est une étape de standardisation des données ou normalisation. Les variables sont divisées par leur écart-type. Ce n'est pas utile dans le cas d'un modèle linéaire élémentaire car la solution est identique mais indispensbale pour beaucoup d'autres méthodes non linéaires (SVM, réseaux de neurones, modèles avec pénalisation). Cette étape est donc concrètement systématiquement excéutée pour éviter des soucis. *Attention*, les mêmes paramètres  (moyennes, écarts-types) estimés sur l'échantillon d'apprentissage sont utilisés pour normaliser l'échantillon test. 

In [ ]:
from sklearn.preprocessing import StandardScaler  

In [ ]:
scaler = StandardScaler()  
scaler.fit(pd.concat([X_train, X_test], axis = 'rows')) 

Il est préférable de centrer et de réduire les données lors de l'utilisation de modèles tels que les réseaux de neurones et les modèles linéaires. La réduction de l'échelle des variables facilitera la convergence du modèle.

In [ ]:
X_train = pd.DataFrame(
    scaler.transform(X_train), 
    columns = df.columns, 
    index = X_train.index
) 

X_test = pd.DataFrame(
    scaler.transform(X_test), 
    columns = df.columns, 
    index = X_test.index
)

In [ ]:
X_train.head()

In [ ]:
df

### 4.2 Modèles linéaires

Toutes les variables ne contribuent pas à expliquer la cible `O3obs`. Pour réduire le bruit du modèle, nous utilisons des méthodes de sélection de variables et tentons de retirer les variables qui n'apportent pas d'information. Un moyen efficace d'identifier les variables qui n'aident pas à modéliser notre variable cible est d'utiliser la régularisation.

Nous utilisons la [pénalisation Lasso](http://wikistat.fr/pdf/st-m-app-select.pdf) dans le cadre de cette exemple. Nous pourrions aussi `retirer / ajouter` successivement les variables afin de mesurer leur contribution à la modélisation de la variable cible.

**Q** Quel autre type de pénalisation est aussi utilisée en régression?

**Q** Quelle la méthode qui combine les deux?

A titre de comparaison, on trace la prévision de la concentration de l'échantillon test par la seule valeur du modèle *Mocage* ainsi que les résidus à ce modèle fonction de la valeur prédite (Mocage)

In [ ]:
X_train.head()

In [ ]:
plt.plot(X_train["MOCAGE"], y_train, "o")
plt.xlabel("Mocage")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
f"R2 Score: {r2_score(y_train,X_train['MOCAGE'])}"

In [ ]:
plt.plot(df["MOCAGE"], y, "o")
plt.xlabel("Mocage")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
plt.plot(df["MOCAGE"], df["MOCAGE"] - y, "o")
plt.xlabel("Mocage")
plt.ylabel("Residus")
plt.show()

**Q** Commenter la qualité de ces résidus.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
f"Erreur quadratique moyenne: {mean_squared_error(df['MOCAGE'], y)}"

In [ ]:
# Le coefficient de détermination 
# peut être négatif en prévision avec un mauvais modèle, 
# est nul si la prévision est constante égale à la moyennne
f"R2 score: {r2_score(df['MOCAGE'], y)}"

#### [Régression linéaire](http://wikistat.fr/pdf/st-m-app-select.pdf) ou modèle gaussien

Comparer cette prévision déterministe (équation de Navier et Stockes) par l'adaptation statistique la plus élémentaire. Il s'agit d'une régression avec choix de modèle par régularisation avec une pénalisation lasso. 

**Question:** Quelles est la valeur par défaut du paramètre de pénalisation Lasso?.

In [ ]:
from sklearn import linear_model

In [ ]:
model = linear_model.Lasso()
model.fit(X_train,y_train)
f"Erreur quadratique moyenne: {mean_squared_error(y_true = y_test, y_pred = model.predict(X_test))}"

In [ ]:
from sklearn.metrics import r2_score
print("R2=",r2_score(y_test, model.predict(X_test)))

Le paramètre de pénalisation lasso est optimisé par validation croisée.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# grille de valeurs du paramètre alpha à optimiser
grid =[{"alpha":[0.05,0.1,0.2,0.3,0.4,0.5,1]}]

model = GridSearchCV(linear_model.Lasso(), grid, cv = 5)

model = model.fit(X_train, y_train)

# paramètre optimal
model.best_params_["alpha"]

f"Meilleur R2 = {model.best_score_}, Meilleur paramètre = {model.best_params_}"

**Question** Quelle validation croisée est exécutée?

Prévision avec la valeur optimale de `alpha` puis calcul et tracé des résidus.

In [ ]:
y_pred = model.predict(X_test)
print(f"MSE: {mean_squared_error(y_pred, y_test):4f}")
print(f"R2 : {r2_score(y_test, y_pred):4f}")

In [ ]:
plt.plot(y_pred, y_test,"o")
plt.xlabel(u"O3 Prédite")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
plt.plot(y_pred, y_test - y_pred,"o")
plt.xlabel(u"Prédites")
plt.ylabel(u"Résidus")
plt.hlines(0,40,220)
plt.show()

**Q** Comparer ces résidus avec ceux précédents (mocage) et noter l'amélioration. 

**Q** Commenter la forme du nuage et donc la validité du modèle. 

L'interprétation nécessite de connaître les valeurs des coefficients du modèle alors que l'objet `regLassOpt` issu de `GridSearchCV` ne retient pas les paramètres estimés. Il faut donc le ré-estimer avec la valeur optimale du paramètre de pénalisation si l'on souhaite afficher ces coefficients.

In [ ]:
# Coefficients
model = linear_model.Lasso(alpha=model.best_params_['alpha'])
model = model.fit(X_train, y_train)
model.coef_

In [ ]:
coef = pd.Series(model.coef_, index = X_train.columns)
print("Lasso conserve " + str(sum(coef != 0)) + 
      " variables et en supprime " +  str(sum(coef == 0)))

In [ ]:
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title(u"Coefficients du modèle lasso")

**Question:** Noter les conséquences de la pénalisation; interpréter l'effet de chaque variable sur la concentration en ozone.

Note: Il serait intéressant de compléter ces résultats avec les p-valeurs qui ne sont pas fournies par Sklearn.

Le graphe quivant permet d'identifier les bonnes et mauvaises prévisions de dépassement du seuil légal, ici fixé à $ 150 \mu g $.

In [ ]:
plt.plot(y_pred, y_test,"o")
plt.xlabel(u"Valeurs prédites")
plt.ylabel(u"O3 observée")
plt.hlines(150,50,300)
plt.vlines(150,0,300)
plt.show()

In [ ]:
# Dénombrement des erreurs par
# matrice de confusion
table=pd.crosstab(y_pred>150, y_test>150)
print(table)

**Question** Observer l'asymétrie de cette matrice. A quoi est-elle due au moins en partie ?

*Scikit-learn* propose d'autres procédures d'optimisation du paramètre de régularisation lasso par validation croisée en régression; `lassoCV` utilise un algorithme de *coordinate descent*, sans calcul de dérivée puisque la norme *l1* n'est pas dérivable, tandis que `lassoLarsCV` est basée sur l'algorithme de *least angle regression*. Ces fonctions permettent de tracer également les *chemins de régularisation*. Voici l'exemple de `lassoCV` qui offre plus d'options.

In [ ]:
from sklearn.linear_model import LassoCV, LassoLarsCV

In [ ]:
model = LassoCV(
    cv = 5, 
    alphas = np.array(range(1,50,1)) / 20.,
    random_state=42
)

model = model.fit(X_train, y_train)

In [ ]:
m_log_alphas = - np.log10(model.alphas_)

plt.figure()
# ymin, ymax = 2300, 3800
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='MSE moyen', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha: optimal par VC')

plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('MSE')
plt.title('MSE de chaque validation: coordinate descent ')
plt.show()

**Question:** Vérifier que c'est bien la même valeur optimale que celle précédemment trouvée.

Tracés des chemins de régularisation.

In [ ]:
from itertools import cycle
from sklearn.linear_model import lasso_path

In [ ]:
alphas_lasso, coefs_lasso, _ = lasso_path(
    X_train,
    y_train, 
    alphas = np.array(range(1,50,1))/20.,
)

plt.figure()

ax = plt.gca()

styles = cycle(['-', '--', '-.', ':'])

neg_log_alphas_lasso = - np.log10(alphas_lasso)

for coef_l, s in zip(coefs_lasso, styles):
    
    l1 = plt.plot(neg_log_alphas_lasso, coef_l, linestyle=s, c='b')
                  
plt.xlabel('-Log(alpha)')
plt.ylabel('Coefficients')
plt.show()

#### [Régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf) ou modèle binomial

La même démarche est déroulée mais en modélisant directement la variable binaire Yb de dépassement ou non du seuil. Il s'agit d'une régression logistique avec toujours une pénalisation Lasso pour opérer une sélection de variables.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Optimisation du paramètre de pénalisation
# grille de valeurs

grid = [
    {"C":[1,1.2,1.5,1.7,2,3,4]}
]

logistic_regression = GridSearchCV(
    LogisticRegression(penalty="l1",solver='liblinear'), 
    grid, 
    cv = 5
)

logistic_regression = logistic_regression.fit(X_train, y_binary_train) 

f"Meilleur score: {1. - logistic_regression.best_score_}, meilleur paramètre: {logistic_regression.best_params_}"

In [ ]:
# erreur sur l'échantillon test
1 - logistic_regression.score(X_test, y_binary_test)

Le modèle "optimal"  obtenu est utilisé pour prédire l'échantillon test et estimer ainsi, sans biais, une erreur de prévision. 

La matrice de confusion croise les dépassements de seuils prédits avec ceux effectivement observés. 

In [ ]:
# Prédiction
y_pred = logistic_regression.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

L'interprétation du modèle est basée sur les valeurs des coefficients avec les mêmes difficultés ou restrictions que pour la régression. Attention, `GridSearch` ne retient pas les coefficients, il faut les ré-estimer.

In [ ]:
# Coefficients
logistic_regression = LogisticRegression(
    penalty = "l1",
    solver = 'liblinear',
    C = logistic_regression.best_params_['C']
)

coef = logistic_regression.fit(X_train, y_binary_train).coef_

coef = pd.Series(coef[0], index = X_train.columns).sort_values()

coef

In [ ]:
f"Lasso conserve {sum(coef != 0)} variables et en supprime {sum(coef == 0)}"

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)

coef.plot(kind = "barh")

plt.title(u"Coefficients du modèle lasso")

**Q** Interpréter l'effet des variables retenues.

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
logistic_regression = LogisticRegression(
    penalty = "l1",
    solver = 'liblinear',
    C = 3,
)

logistic_regression = logistic_regression.fit(X_train, y_binary_train)

y_pred = logistic_regression.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_binary_test, y_pred[:,1])

plt.plot(fpr, tpr, lw=1)
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.show()

**Q** Commenter la courbe ROC à propos du choix de la valeur seuil.

### 4.3 [K plus proches voisins](http://wikistat.fr/pdf/st-m-app-add.pdf)

Voici un cas d'application d'analyses discriminantes [non paramétriques](http://scikit-learn.org/stable/modules/neighbors.html), celles [paramétriques](http://scikit-learn.org/stable/modules/lda_qda.html) (gaussienes) linéaires et quadratiques sont également présentes dans *scikit-learn* mais laissées en exercice.

Le paramètre de compléxité ($k$) est optimisé sur une grille prédéfinie en minimisant l'erreur estimée par validation croisée; scikit-learn propose de nombreuses options de validation croisée. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Optimisation de k
# grille de valeurs
grid = [{"n_neighbors":list(range(1,15))}]

knn = GridSearchCV(
    KNeighborsClassifier(),
    grid,
    cv = 5,
)

knn = knn.fit(X_train, y_binary_train) 

# paramètre optimal
knn.best_params_["n_neighbors"]

f"Meilleur score = {1.- knn.best_score_:f}, meilleur paramètre: {knn.best_params_}"

In [ ]:
# Estimation de l'erreur de prévision sur l'échantillon test
1 - knn.score(X_test, y_binary_test)

In [ ]:
# Prévision de l'échantillon test
y_pred = knn.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

**Exercice** Compléter les résultats en utilisant la fonction [KNeighborsRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) pour modéliser la concentration; optimiser $k$, calculer la prévision de l'échantillon test, tracer le graphe des résidus, calculer le MSE sur l'échantillon test.

# <FONT COLOR="Green">Compléments: Méthodes d'Apprentissage </font> du [Module d'Immersion](https://github.com/Certificat-sciences-des-donnees-bigdata/Module-immersion)

### 4.4. [Arbre binaire de décision](http://wikistat.fr/pdf/st-m-app-cart.pdf)

Les [arbres binaires de décision](http://scikit-learn.org/stable/modules/tree.html) : discrimination ou régression, sont bien implémentés dans *scikit-learn* mais avec une insuffisance pour leur élagage. Ce n'est pas une *pénalisation* de la *complexité*, et donc précisément le nombre de feuilles qui est optimisé, mais la profondeur globale de l'arbre au risque d'élaguer, à une profondeur donnée, des feuilles importantes ou de conserver des feuilles ambigües.

Comme précédemment, la validation croisée permet d'optimiser le paramètre sur une grille.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Optimisation de la profondeur de l'arbre
grid = [{"max_depth":list(range(2,10))}]

decision_tree = GridSearchCV(
    DecisionTreeClassifier(), 
    grid,
    cv = 10,
)

decision_tree = decision_tree.fit(X_train, y_binary_train)

# paramètre optimal
f"Meilleur score = {1. - decision_tree.best_score_:f}, meilleur paramètre: {decision_tree.best_params_}"

In [ ]:
# Estimation de l'erreur de prévision
1 - decision_tree.score(X_test, y_binary_test)

In [ ]:
# prévision de l'échantillon test
y_pred = decision_tree.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

Autre difficulté dans la représentation d'un arbre de décision binaire. Le logiciel conseillé (Graphviz) semble délicat d'installation et d'utilisation pour un néophyte. Il est possible de lister la construction des noeuds avec quelques [lignes de commande.](http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py)

In [ ]:
decision_tree = DecisionTreeClassifier(
    max_depth = decision_tree.best_params_['max_depth']
)

decision_tree.fit(X_train, y_binary_train)

In [ ]:
from sklearn import tree

In [ ]:
plt.figure(figsize=(20, 10))
_ = tree.plot_tree(decision_tree, feature_names = X_train.columns, fontsize = 10)
plt.show()

**Q** Que dire de l'interprétation de l'arbre? Comparer les rôles des variables avec le modèle logit.

**Exercice** Compléter les résultats en utilisant la fonction [DecisionTreeRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) pour modéliser concentration; optimiser la profondeur, calculer la prévision de l'échantillon test, tracer les résidus, calculer le MSE sur l'échantillon test.

### 4.5 [Réseau de neurones](http://wikistat.fr/pdf/st-m-app-rn.pdf)

Les réseaux neuronaux (perceptron multicouche) ne sont présents dans le package `Scikit-learn` qu'à partir de la version 0.18. Les méthodes *profondes* (*deep learning*) nécessitent l'installation des librairies [*theano*](http://deeplearning.net/software/theano/) et [*Lasagne*](http://lasagne.readthedocs.io/en/latest/index.html) ou [*theano*](http://deeplearning.net/software/theano/), [*TensorFlow*](https://www.tensorflow.org/versions/r0.11/get_started/os_setup.html) et [*Keras*](https://keras.io/). Ces dernières sont nettement plus complexes à installer, surtout sous Windows. Elles feront l'objet d'un autre tutoriel.

In [ ]:
from sklearn.neural_network import MLPClassifier

Définition des paramètres dont le nombre de neurones et `alpha` qui règle la régularisation par défaut 10-5. Le nombre de neurones est optimisé mais ce peut être `alpha` avec un nombre grand de neurones. Le nombre max d'itérations par défaut (200) semble insuffisant. Il est fixé à 500.

In [ ]:
grid = [{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}]

neural_network = GridSearchCV(
    MLPClassifier(max_iter=1000, warm_start=True, random_state=42),
    grid,
    cv = 10,
)

neural_network = neural_network.fit(X_train, y_binary_train)

# paramètre optimal
f"Meilleur score = {1.- neural_network.best_score_:f}, meilleur paramètre: {neural_network.best_params_}"

In [ ]:
# Estimation de l'erreur de prévision sur le test
1 - neural_network.score(X_test, y_binary_test)

In [ ]:
# prévision de l'échantillon test
y_pred = neural_network.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

**Exercice** Remplacer ensuite la fonction MLPClassifier par celle [MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) de régression. Optimiser le paramètre, calculer la prévision, les résidus, le MSE.

### 4.6 [Forêts aléatoires](http://wikistat.fr/pdf/st-m-app-agreg.pdf)

La librairie *randomForest* de R utilise le programme historique développé par [Breiman et Cutler](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_software.htm)(2001) et interfacé  par [Liaw et Wiener](https://cran.r-project.org/web/packages/randomForest/randomForest.pdf). Cette interface est toujours mise à jour mais il n'est pas sûr que le programme original continue d'évoluer depuis 2004. Pour des tailles importantes d'échantillons, quelques milliers, cette implémentation atteint des temps d'exécution rédhibitoires (cf. cet [exemple](https://github.com/wikistat/Ateliers-Big-Data/blob/master/2-MNIST/Atelier-MNIST-R.ipynb)) au contraire de celle en Python dont gestion mémoire et capacité de parallélisation ont été finement optimisées par [Louppe et al.](http://fr.slideshare.net/glouppe/accelerating-random-forests-in-scikitlearn)(2014). 

De même que le boosting, deux fonctions  de forêt sont proposés dans [scikit-learn](http://scikit-learn.org/stable/modules/ensemble.html) ; une pour la régression et une pour la classification ainsi qu'une version "plus aléatoire". Par rapport à la version originale de R, moins d'options sont proposées mais l'utilisation de base est très similaire avec le même jeu de paramètres.

**Q** Identifier les paramètres, les valeurs par défaut.

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
# définition des paramètres
random_forest = RandomForestClassifier(
    n_estimators = 500,
    criterion = 'gini', 
    max_depth = None,
    min_samples_split = 2, 
    min_samples_leaf = 1, 
    max_features = 'auto', 
    max_leaf_nodes = None,
    bootstrap = True, 
    oob_score = True
)
# apprentissage
random_forest = random_forest.fit(X_train,y_binary_train)

1 - random_forest.oob_score_

Comparer l'erreur out-of-bag ci-dessus avec celle sur l'échantillon test.

In [ ]:
# erreur de prévision sur le test
1 - random_forest.score(X_test,y_binary_test)

Optimisation par validation croisée du nombre de variables tirés aléatoirement lors de la construction de chaque noeud. 

In [ ]:
grid = [{"max_features":list(range(2,10,1))}]

random_forest = GridSearchCV(
    RandomForestClassifier(n_estimators=100, random_state=42),
    grid,
    cv = 5,
)

random_forest = random_forest.fit(X_train, y_binary_train)

# paramètre optimal
f"Meilleur score: {1. - random_forest.best_score_:f}, meilleur paramètre: {random_forest.best_params_}"

Plusieurs exécutions, rendues aléatoires par la validation croisée, peuvent conduire à des valeurs "optimales" différentes de ce paramètre sans pour autant nuire à la qualité de prévision sur l'échantillon test.

In [ ]:
# erreur de prévision sur le test
1 - random_forest.score(X_test,y_binary_test)

**Exercice** Tester différentes valeurs de *min_samples_split* de celle trouvée optimale. Conclusion sur la sensibilité de l'optimisation de ce paramètre ?

In [ ]:
# prévision
y_pred = random_forest.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

Comme avec R, il est possible de calculer un indicateur d'importance des variables pour aider à une forme d'interprétation. Celui-ci dépend de la position de la variable dans l'arbre et correspond donc au *mean decrease in Gini index* de R plutôt qu'au *mean descrease in accuracy*. La forêt doit être réestimée car GridSearch ne connaît pas le paramètre d'importance.

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100,max_features=2)
random_forest = random_forest.fit(X_train, y_binary_train)

# Importance des variables
importances = pd.DataFrame(
    random_forest.feature_importances_, 
    index = X_train.columns, 
    columns = ["importance"]
).sort_values(ascending = False, by = "importance")

importances

In [ ]:
importances.plot.bar()

**Q** Comparer les importances des variables et les sélections opérées précédemment. 

**Exercice** Remplacer ensuite la fonction RandomForestClassifier par celle [RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) de régression. Optimiser le paramètre, calculer la prévision, les résidus, le MSE.

**Exercice** Expérimenter également le boosting sur ces données en exécutant la fonction [GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier) opérant l'agorithme de *gradient tree boosting*. 

**Remarque:** Une version "améliorée" de *boosting* mieux paralélisée et incluant d'autres paramètres (pénalisation), est  proposé dans le package: [`XGBoost`](https://xgboost.readthedocs.io/en/latest/build.html#python-package-installation) qui peut être utilisé à partir de Python mais aussi R, Julia ou Java. Nénamoins le choix est fait d'arrêter l'acharnement sur ces données; `XGBoost` est testé en python sur un autre jeu de données. 

### 4.7 [*Support Vector Machine*](http://wikistat.fr/pdf/st-m-app-svm.pdf)

De nombreux paramètres sont associés à cette méthode. La liste est à consulter dans la [documentation](http://scikit-learn.org/stable/modules/svm.html) en ligne.

L'optimisation de la pénalisation (paramètre C) est recherchée sur une grille par validation croisée. Remarque: il serait nécessaire d'optimiser également la valeur du coefficient *gamma* lié au noyau gaussien ("écart-type").

Il est souvent nécessqire de normaliser des données avant d'opérer les SVM mais cela ne semble pas nécessaire ici.

In [ ]:
from sklearn.svm import SVC

In [ ]:
grid = [{"C":[0.4,0.5,0.6,0.8,1,1.4]}]

svm = GridSearchCV(
    SVC(random_state = 42),
    grid,
    cv=10
)

svm = svm.fit(X_train, y_binary_train)

# paramètre optimal
f"Meilleur score: {1.- svm.best_score_:f}, meilleur paramètre: {svm.best_params_}"

In [ ]:
# erreur de prévision sur le test
1 - svm.score(X_test, y_binary_test)

In [ ]:
# prévision de l'échantillon test
y_pred = svm.predict(X_test)

# matrice de confusion
pd.crosstab(y_pred, y_binary_test)

**Exercice** Comme précédemment, remplacer ensuite la fonction SVC par celle [SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) de régression. Optimiser le paramètre, calculer la prévision, les résidus; le MSE.

## 5.1 Synthèse: comparaison des méthodes

### 5.1 Courbes ROC

Dans toute méthode, la prévision de dépassement ou non est associée au choix d'un seuil qui est par défaut 0.5. L'optimisaiton de ce seuil dépend des coûts respectifs associés aux faux positifs et aux faux négatifs qui ne sont pas nécessairement égaux. La courbe ROC permet de représenter l'influence de ce seuil sur les taux de faux positifs et vrais positifs.  

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
methods = {
    "random_forest": random_forest,
    "neural_netork": neural_network,
    "decision_tree": decision_tree,
    "knn": knn,
    "logistic_regression": logistic_regression,
}

In [ ]:
for method, model in methods.items():
    
    model = model.fit(X_train, y_binary_train)
    
    probas_ = model.predict_proba(X_test)

    fpr, tpr, thresholds = roc_curve(y_binary_test, probas_[:,1])
    
    plt.plot(fpr, tpr, lw = 1,label = method)
    
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
plt.show()

**Q22** Le critère d'AUC (aire sous la courbe) permet-il d'ordonner les courbes et donc les méthodes? 

C'est à un taux de faux positif admissible et donc à valeur de seuil fixé qu'il faut choisir la méthode d'apprentissage à privilégier. 

### Conclusion sur l'apprentissage
**Q** Comparer les qualités de prévision d'occurrence d'un pic d'ozone: MOCAGE *vs.* adaptation statistique

**Q** Quel méthode d'apprentissage retenir en termes de qualité de prévision?

**Q** Quel méthode d'apprentissage retenir pour un meilleur compromis prévision / interprétation.

###  5.2 Zoom sur la validation croisée, Kfold cross-validation

Nous avons utilisé la méthode `GridSearchCV` pour rechercher les meilleurs paramètres des différents modèles. La méthode `GridSearchCV` effectue une validation croisée pour chaque configuration de la grille de paramètres indiquée par l'utilisateur. 

La validation croisée permet d'évaluer le modèle sur l'ensemble des données. Le meilleur modèle est celui qui, en moyenne, généralise le mieux sur les données de validation. 

Pour effectuer une validation croisée, le jeu de données doit être divisé en `k` parties (k folds) de taille identique. Ensuite, nous devons entraîner le modèle sur `k-1` parties des données et valider le modèle sur la partie que l'on a exclu de l'entrainement. Il faut répéter l'opération `k` fois de façons à évaluer le modèle sur l'ensemble des `k` parties du dataset. Il est parfois utile de mélanger les données avant de construire les plis de validation. 

Si nous décidons d'effectuer une validation croisée à 5 plis, nous devrons définir les différents plis du dataset qui contiendront chacun 20% des données :

<table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="background-color:#ffccc9;border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Fold 1 (20% observations)</th></tr></thead><tbody><tr><td style="background-color:#ffce93;border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Fold 2 <span style="font-weight:normal;font-style:normal;text-decoration:none">(20% observations)</span></td></tr><tr><td style="background-color:#fffc9e;border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Fold 3 <span style="font-weight:normal;font-style:normal;text-decoration:none">(20% observations)</span></td></tr><tr><td style="background-color:#9aff99;border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Fold 4 <span style="font-weight:normal;font-style:normal;text-decoration:none">(20% observations)</span></td></tr><tr><td style="background-color:#96fffb;border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Fold 5 <span style="font-weight:normal;font-style:normal;text-decoration:none">(20% observations)</span></td></tr></tbody></table>


- Entrainer le modèle sur les parties 2, 3, 4, 5 et mesurer l'erreur `e1` sur la partie 1.

- Entrainer le modèle sur les parties 1, 3, 4, 5 et mesurer l'erreur `e2` sur la partie 2.

- Entrainer le modèle sur les parties 1, 2, 4, 5 et mesurer l'erreur `e3` sur la partie 3.

- Entrainer le modèle sur les parties 1, 2, 3, 5 et mesurer l'erreur `e4` sur la partie 4.

- Entrainer le modèle sur les parties 1, 2, 3, 4 et mesurer l'erreur `e5` sur la partie 5.

L'erreur du modèle est égale à la moyenne des erreurs des plis de validation: `(e1 + e2 + e3 + e4 + e5) / 5`

Si vous avez très peu de données, vous pouvez utiliser la validation croisée `leave-one-out`. C'est un cas particulier de la validation croisée où le nombre de plis `k` est égal au nombre d'observations dans le jeu de données. La méthode `leave-one-out` consiste à entraîner un modèle sur `n-1` observations et à le valider sur une seule observation du jeu de données. L'augmentation de la valeur de `k` est coûteuse en temps d'exécution. 

In [ ]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score


# Intialisation du model
model = RandomForestClassifier(max_depth = 20, n_estimators = 100)
# model = LogisticRegression()
# model = MLPClassifier(max_iter=10000, warm_start=True, random_state=42)

# e1, e2, ...
error = {}

# Découpage du dataset en 5 parties, on mélange les données au préalable.
kfold = model_selection.KFold(n_splits = 5,  shuffle = True)

for split, (fit, val) in enumerate(kfold.split(X_train, y_binary_train)):
    
    # Sélection de l'ensemble des features
    X_fit = X_train.iloc[fit]
    X_val = X_train.iloc[val]
    
    # Sélection des variables cibles
    y_fit = y_binary_train.iloc[fit]
    y_val = y_binary_train.iloc[val]
    
    # Entrainement du modèle
    model = model.fit(X_fit, y_fit)
    
    # Calcul de l'erreur
    error[f"split_{split}"] = accuracy_score(
        y_pred = model.predict(X_val), 
        y_true = y_val,
    )
    
    print(f"Accuracy fold {split}: {error[f'split_{split}']:4f}")

print(f"\nAverage accuracy: {np.mean(list(error.values())):4f}, std: {np.std(list(error.values())):4f}")

In [ ]:
# matrice de confusion
pd.crosstab(model.predict(X_test), y_binary_test)

Note:

L'objectif de la validation est d'estimer l'erreur du modèle lorsqu'il sera appliqué sur le terrain, il est donc nécessaire de simuler au mieux l'environnement du modèle lors de son évaluation. La validation croisée n'est pas toujours la meilleure façon d'estimer cette erreur. Il existe de nombreuses alternatives à la validation croisée. Nous n'étudierons pas ces solutions dans ce cours mais vous pouvez trouver les différentes stratégies de validation sur le site de [sklearn](https://scikit-learn.org/stable/modules/cross_validation.html).

Une liste non exhaustive de stratégies de validation:

- Validation croisée Monte Carlo
- Validation croisée groupée
- Validation croisée stratifiée
- Validation temporelle
- Validation progressive
